In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

import requests
import json
from pandas import json_normalize

import geopandas as gpd
from shapely.geometry import Point

import folium
from folium import plugins
from folium.plugins import HeatMap, MarkerCluster

import h3
from shapely.geometry import Polygon, box, Point  # Import Point from shapely.geometry

In [2]:
taxi_df = pd.read_csv('assets/processed_data/taxi_availability.csv')
bus_df = pd.read_csv('assets/processed_data/bus_volume.csv')
train_df = pd.read_csv('assets/processed_data/train_volume.csv')

In [3]:
taxi_df = taxi_df.drop(columns = 'Unnamed: 0')
taxi_df

,longitude,latitude,weekday_00,weekday_01,weekday_02,weekday_03,weekday_04,weekday_05,weekday_06,weekday_07,...,weekend_14,weekend_15,weekend_16,weekend_17,weekend_18,weekend_19,weekend_20,weekend_21,weekend_22,weekend_23
0,103.625536,1.305826,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,103.667031,1.298071,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,103.670050,1.319960,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,103.678860,1.318840,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,103.680730,1.322490,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1342616,104.001260,1.379150,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1342617,104.001820,1.377500,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1342618,104.002220,1.375970,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1342619,104.002220,1.375970,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
bus_df = bus_df.drop(columns = 'Unnamed: 0')
bus_df

,latitude,longitude,weekday_00,weekday_01,weekday_02,weekday_03,weekday_04,weekday_05,weekday_06,weekday_07,...,weekend_14,weekend_15,weekend_16,weekend_17,weekend_18,weekend_19,weekend_20,weekend_21,weekend_22,weekend_23
0,1.394588,103.751869,0,0,0,0,0,0,0,0,...,0,0,0,203,0,0,0,0,0,0
1,1.394588,103.751869,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1.394588,103.751869,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1.394588,103.751869,18,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1.394588,103.751869,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,80,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190935,1.359102,103.767052,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,295,0,0
190936,1.359102,103.767052,0,0,0,0,0,0,0,0,...,0,0,0,0,0,335,0,0,0,0
190937,1.359102,103.767052,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
190938,1.359102,103.767052,59,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
train_df = train_df.drop(columns = 'Unnamed: 0')
train_df

,geometry,latitude,longitude,weekday_00,weekday_05,weekday_06,weekday_07,weekday_08,weekday_09,weekday_10,...,weekend_14,weekend_15,weekend_16,weekend_17,weekend_18,weekend_19,weekend_20,weekend_21,weekend_22,weekend_23
0,POINT (103.8628597696374 1.27133671132917),1.271337,103.862860,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,POINT (103.8628597696374 1.27133671132917),1.271337,103.862860,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,287,0
2,POINT (103.8628597696374 1.27133671132917),1.271337,103.862860,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,POINT (103.8628597696374 1.27133671132917),1.271337,103.862860,0,0,0,0,902,0,0,...,0,0,0,0,0,0,0,0,0,0
4,POINT (103.8628597696374 1.27133671132917),1.271337,103.862860,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6696,POINT (103.9322346236246 1.336607829810492),1.336608,103.932235,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6697,POINT (103.9322346236246 1.336607829810492),1.336608,103.932235,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6698,POINT (103.9322346236246 1.336607829810492),1.336608,103.932235,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6699,POINT (103.9322346236246 1.336607829810492),1.336608,103.932235,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Spatial Indexing

Define the list of functions that are useful for:
- Initiating and expanding hexagon
- Creating latitude, longitude, and h3 addresses of each hexagon
- Filter the hexagon based on pre-defined boundary
- Visualize hexagons in the folium map

In [6]:
# Define a function to visualize the generated hexagons in Folium
def visualize_hexagons(hexagons, color="red", folium_map=None, zoom_start=11):
    polylines = []
    lat = []
    lng = []
    for hex in hexagons:
        polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
        # Flatten polygons into loops.
        outlines = [loop for polygon in polygons for loop in polygon]
        polyline = [outline + [outline[0]] for outline in outlines][0]
        for vertex in polyline:
            lat.append(vertex[0])
            lng.append(vertex[1])
        polylines.append(polyline)
    
    if folium_map is None:
        m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=13, tiles='cartodbpositron')
    else:
        m = folium_map
    for polyline in polylines:
        my_PolyLine=folium.PolyLine(locations=polyline,weight=2,color='blue')
        m.add_child(my_PolyLine)
    return m

# Function to expand the hexagon
def expand_h3_address(h3_address, k):
    hexagons = h3.k_ring(h3_address, k)
    return hexagons

def create_address_dataframe(hexagons):
    # Initialize lists to store latitude, longitude, and H3 address
    latitudes = []
    longitudes = []
    h3_addresses = []
    
    # Iterate over each hexagon
    for hexagon in hexagons:
        # Get the latitude and longitude of the hexagon's center
        lat, lng = h3.h3_to_geo(hexagon)
        # Append the values to the lists
        latitudes.append(lat)
        longitudes.append(lng)
        h3_addresses.append(hexagon)
    
    # Create DataFrame
    address_data = {
        'h3_address': h3_addresses,
        'latitude': latitudes,
        'longitude': longitudes,
        
    }
    address_df = pd.DataFrame(address_data)
    return address_df

# Function to filter the hexagon based on the boundary
def filter_hexagons_by_boundary(hexagons, boundary_polygon):
    filtered_hexagons = []
    for hexagon in hexagons:
        lat, lng = h3.h3_to_geo(hexagon)
        if boundary_polygon.contains(Point(lng, lat)):  # Corrected the order of lat and lng
            filtered_hexagons.append(hexagon)
    return filtered_hexagons

Call out the function with summary as follows
- Initiate the hexagon from center of Singapore, which is the coordinate of 1.3521, 103.8198
- Define the boundary of Singapore to limit the hexagon formation
- Print the list of hexagons within the boundary
- Visualize in the Folium

In [7]:
h3_address = h3.geo_to_h3(1.3521, 103.8198, 7)

initial_hexagon = [h3_address]
m = visualize_hexagons(initial_hexagon, color="blue", zoom_start=13)
m

In [8]:
expanded_hexagons = expand_h3_address(h3_address, 1)  # Expand by 1 rings

m = visualize_hexagons(expanded_hexagons, color="blue", zoom_start=13)
m

In [9]:
h3_address = h3.geo_to_h3(1.3521, 103.8198, 7)  # lat, lng, hex resolution
expanded_hexagons = expand_h3_address(h3_address, 50)  # Expand by 50 rings

# Define your boundary as a rectangle
min_lat, min_lng, max_lat, max_lng = 1.20, 103.600, 1.470, 104.0300
boundary_polygon = box(min_lng, min_lat, max_lng, max_lat)

# Filter expanded hexagons by the boundary
filtered_hexagons = filter_hexagons_by_boundary(expanded_hexagons, boundary_polygon)

# Create a DataFrame of the filtered hexagons' addresses
address_df = create_address_dataframe(filtered_hexagons)
print(address_df)

# Visualize the filtered hexagons on a map
map_visual = visualize_hexagons(filtered_hexagons, color="blue")
map_visual

          h3_address  latitude   longitude
0    876520c18ffffff  1.290950  103.628706
1    876526aeaffffff  1.289424  103.943911
2    876520c82ffffff  1.415494  103.707587
3    87652632effffff  1.430500  104.006687
4    876520c93ffffff  1.460598  103.720903
..               ...       ...         ...
241  87652636affffff  1.405831  103.831062
242  876526ac6ffffff  1.323753  103.996124
243  876526305ffffff  1.453036  104.013356
244  876520db4ffffff  1.255092  103.891696
245  876520d8affffff  1.303865  103.758838

[246 rows x 3 columns]


From the hexagon address, create a GeoDataFrame containing the polygon geometries

In [10]:
geometry = []
for index, row in address_df.iterrows():
    vertices = h3.h3_to_geo_boundary(row['h3_address'], True)
    polygon_geom = Polygon(vertices)
    geometry.append(polygon_geom)

# Create a GeoDataFrame with Polygon geometries
hexagon_gdf = gpd.GeoDataFrame(address_df, geometry=geometry)
hexagon_gdf

,h3_address,latitude,longitude,geometry
0,876520c18ffffff,1.290950,103.628706,"POLYGON ((103.64169 1.28738, 103.63852 1.30043..."
1,876526aeaffffff,1.289424,103.943911,"POLYGON ((103.95687 1.28585, 103.95372 1.29890..."
2,876520c82ffffff,1.415494,103.707587,"POLYGON ((103.72057 1.41192, 103.71741 1.42498..."
3,87652632effffff,1.430500,104.006687,"POLYGON ((104.01965 1.42692, 104.01650 1.43998..."
4,876520c93ffffff,1.460598,103.720903,"POLYGON ((103.73389 1.45702, 103.73072 1.47008..."
...,...,...,...,...
241,87652636affffff,1.405831,103.831062,"POLYGON ((103.84404 1.40225, 103.84088 1.41531..."
242,876526ac6ffffff,1.323753,103.996124,"POLYGON ((104.00908 1.32018, 104.00594 1.33323..."
243,876526305ffffff,1.453036,104.013356,"POLYGON ((104.02632 1.44946, 104.02317 1.46251..."
244,876520db4ffffff,1.255092,103.891696,"POLYGON ((103.90466 1.25152, 103.90151 1.26457..."


From the taxi availability data, create a GeoDataFrame containing the point geometry

In [11]:
def create_geodataframe(df):
    # Create geometry column
    geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]
    
    # Create GeoDataFrame with geometry column
    gdf = gpd.GeoDataFrame(df, geometry=geometry)
    
    # Move the geometry column to the 3rd position
    cols = list(gdf.columns)
    cols.insert(2, cols.pop(cols.index('geometry')))
    gdf = gdf[cols]
    
    return gdf

In [12]:
taxi_gdf = create_geodataframe(taxi_df)
taxi_gdf

,longitude,latitude,geometry,weekday_00,weekday_01,weekday_02,weekday_03,weekday_04,weekday_05,weekday_06,...,weekend_14,weekend_15,weekend_16,weekend_17,weekend_18,weekend_19,weekend_20,weekend_21,weekend_22,weekend_23
0,103.625536,1.305826,POINT (103.62554 1.30583),1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,103.667031,1.298071,POINT (103.66703 1.29807),1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,103.670050,1.319960,POINT (103.67005 1.31996),1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,103.678860,1.318840,POINT (103.67886 1.31884),1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,103.680730,1.322490,POINT (103.68073 1.32249),1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1342616,104.001260,1.379150,POINT (104.00126 1.37915),0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1342617,104.001820,1.377500,POINT (104.00182 1.37750),0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1342618,104.002220,1.375970,POINT (104.00222 1.37597),0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1342619,104.002220,1.375970,POINT (104.00222 1.37597),0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
taxi_gdf.iloc[:,:6]

,longitude,latitude,geometry,weekday_00,weekday_01,weekday_02
0,103.625536,1.305826,POINT (103.62554 1.30583),1,0,0
1,103.667031,1.298071,POINT (103.66703 1.29807),1,0,0
2,103.670050,1.319960,POINT (103.67005 1.31996),1,0,0
3,103.678860,1.318840,POINT (103.67886 1.31884),1,0,0
4,103.680730,1.322490,POINT (103.68073 1.32249),1,0,0
...,...,...,...,...,...,...
1342616,104.001260,1.379150,POINT (104.00126 1.37915),0,0,0
1342617,104.001820,1.377500,POINT (104.00182 1.37750),0,0,0
1342618,104.002220,1.375970,POINT (104.00222 1.37597),0,0,0
1342619,104.002220,1.375970,POINT (104.00222 1.37597),0,0,0


In [14]:
bus_gdf = create_geodataframe(bus_df)
bus_gdf.head()

,latitude,longitude,geometry,weekday_00,weekday_01,weekday_02,weekday_03,weekday_04,weekday_05,weekday_06,...,weekend_14,weekend_15,weekend_16,weekend_17,weekend_18,weekend_19,weekend_20,weekend_21,weekend_22,weekend_23
0,1.394588,103.751869,POINT (103.75187 1.39459),0,0,0,0,0,0,0,...,0,0,0,203,0,0,0,0,0,0
1,1.394588,103.751869,POINT (103.75187 1.39459),0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1.394588,103.751869,POINT (103.75187 1.39459),0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1.394588,103.751869,POINT (103.75187 1.39459),18,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1.394588,103.751869,POINT (103.75187 1.39459),0,0,0,0,0,0,0,...,0,0,0,0,0,0,80,0,0,0


In [15]:
train_gdf = create_geodataframe(train_df)
train_gdf.head()

,latitude,longitude,geometry,weekday_00,weekday_05,weekday_06,weekday_07,weekday_08,weekday_09,weekday_10,...,weekend_14,weekend_15,weekend_16,weekend_17,weekend_18,weekend_19,weekend_20,weekend_21,weekend_22,weekend_23
0,1.271337,103.86286,POINT (103.86286 1.27134),0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1.271337,103.86286,POINT (103.86286 1.27134),0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,287,0
2,1.271337,103.86286,POINT (103.86286 1.27134),0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1.271337,103.86286,POINT (103.86286 1.27134),0,0,0,0,902,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1.271337,103.86286,POINT (103.86286 1.27134),0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
hexagon_taxi = hexagon_gdf.copy()

# Iterate over the first three columns of gdf_taxi
for i in taxi_gdf.columns[3:]:
    print(f"Processing column '{i}'")
    
    counts = []  # Initialize an empty list to store the counts for the current column
    
    # Iterate through each polygon in gdf_hexagon and count the number of points inside for the current column
    for index, polygon in hexagon_taxi.iterrows():
        
        # Filter rows in gdf_taxi_may where the current column equals 1, then count points within the current polygon
        count = taxi_gdf[taxi_gdf[i] == 1]['geometry'].within(polygon['geometry']).sum()
        counts.append(count)  # Append the count to the list of counts
    
    # Add the counts as a new column in gdf_hexagon with a column name based on the current column in gdf_taxi_may
    hexagon_taxi[i] = counts

Processing column 'weekday_00'
Processing column 'weekday_01'
Processing column 'weekday_02'
Processing column 'weekday_03'
Processing column 'weekday_04'
Processing column 'weekday_05'
Processing column 'weekday_06'
Processing column 'weekday_07'
Processing column 'weekday_08'
Processing column 'weekday_09'
Processing column 'weekday_10'
Processing column 'weekday_11'
Processing column 'weekday_12'
Processing column 'weekday_13'
Processing column 'weekday_14'
Processing column 'weekday_15'
Processing column 'weekday_16'
Processing column 'weekday_17'
Processing column 'weekday_18'
Processing column 'weekday_19'
Processing column 'weekday_20'
Processing column 'weekday_21'
Processing column 'weekday_22'
Processing column 'weekday_23'
Processing column 'weekend_00'
Processing column 'weekend_01'
Processing column 'weekend_02'
Processing column 'weekend_03'
Processing column 'weekend_04'
Processing column 'weekend_05'
Processing column 'weekend_06'
Processing column 'weekend_07'
Processi

In [17]:
hexagon_taxi.head()

,h3_address,latitude,longitude,geometry,weekday_00,weekday_01,weekday_02,weekday_03,weekday_04,weekday_05,...,weekend_14,weekend_15,weekend_16,weekend_17,weekend_18,weekend_19,weekend_20,weekend_21,weekend_22,weekend_23
0,876520c18ffffff,1.290950,103.628706,"POLYGON ((103.64169 1.28738, 103.63852 1.30043...",30,30,17,14,21,8,...,3,6,6,8,7,11,19,10,5,3
1,876526aeaffffff,1.289424,103.943911,"POLYGON ((103.95687 1.28585, 103.95372 1.29890...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,876520c82ffffff,1.415494,103.707587,"POLYGON ((103.72057 1.41192, 103.71741 1.42498...",3,1,3,2,0,2,...,1,0,2,0,1,0,0,2,6,1
3,87652632effffff,1.430500,104.006687,"POLYGON ((104.01965 1.42692, 104.01650 1.43998...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,876520c93ffffff,1.460598,103.720903,"POLYGON ((103.73389 1.45702, 103.73072 1.47008...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
# Ensure hexagon_gdf_copy is a copy of hexagon_gdf to avoid modifying the original dataframe
hexagon_bus = hexagon_gdf.copy()

# Iterate over the columns starting from the fourth column (index 3) in taxi_gdf
for i in bus_gdf.columns[3:]:
    print(f"Processing column '{i}'")
    
    sums = []  # Initialize an empty list to store the sums for the current column
    
    # Iterate through each polygon in hexagon_gdf and sum the values within the current polygon
    for index, polygon in hexagon_bus.iterrows():
        
        # Filter rows in taxi_gdf to only include points within the current polygon
        filtered_bus = bus_gdf[bus_gdf['geometry'].within(polygon['geometry'])]
        
        # Sum the values of the current column within the filtered_taxi dataframe
        value_sum = filtered_bus[i].sum()
        sums.append(value_sum)  # Append the sum to the list of sums
    
    # Add the sums as a new column in hexagon_gdf_copy
    hexagon_bus[i] = sums

Processing column 'weekday_00'
Processing column 'weekday_01'
Processing column 'weekday_02'
Processing column 'weekday_03'
Processing column 'weekday_04'
Processing column 'weekday_05'
Processing column 'weekday_06'
Processing column 'weekday_07'
Processing column 'weekday_08'
Processing column 'weekday_09'
Processing column 'weekday_10'
Processing column 'weekday_11'
Processing column 'weekday_12'
Processing column 'weekday_13'
Processing column 'weekday_14'
Processing column 'weekday_15'
Processing column 'weekday_16'
Processing column 'weekday_17'
Processing column 'weekday_18'
Processing column 'weekday_19'
Processing column 'weekday_20'
Processing column 'weekday_21'
Processing column 'weekday_22'
Processing column 'weekday_23'
Processing column 'weekend_00'
Processing column 'weekend_01'
Processing column 'weekend_02'
Processing column 'weekend_04'
Processing column 'weekend_05'
Processing column 'weekend_06'
Processing column 'weekend_07'
Processing column 'weekend_08'
Processi

In [19]:
hexagon_bus.head()

,h3_address,latitude,longitude,geometry,weekday_00,weekday_01,weekday_02,weekday_03,weekday_04,weekday_05,...,weekend_14,weekend_15,weekend_16,weekend_17,weekend_18,weekend_19,weekend_20,weekend_21,weekend_22,weekend_23
0,876520c18ffffff,1.290950,103.628706,"POLYGON ((103.64169 1.28738, 103.63852 1.30043...",1,0,0,0,0,0,...,1010,922,1232,1669,1348,861,718,386,229,47
1,876526aeaffffff,1.289424,103.943911,"POLYGON ((103.95687 1.28585, 103.95372 1.29890...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,876520c82ffffff,1.415494,103.707587,"POLYGON ((103.72057 1.41192, 103.71741 1.42498...",0,0,0,0,0,0,...,57,60,75,110,71,79,34,18,10,2
3,87652632effffff,1.430500,104.006687,"POLYGON ((104.01965 1.42692, 104.01650 1.43998...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,876520c93ffffff,1.460598,103.720903,"POLYGON ((103.73389 1.45702, 103.73072 1.47008...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
bus_df['weekend_12'].sum()

1979990

In [21]:
# Ensure hexagon_gdf_copy is a copy of hexagon_gdf to avoid modifying the original dataframe
hexagon_train = hexagon_gdf.copy()

# Iterate over the columns starting from the fourth column (index 3) in taxi_gdf
for i in train_gdf.columns[3:]:
    print(f"Processing column '{i}'")
    
    sums = []  # Initialize an empty list to store the sums for the current column
    
    # Iterate through each polygon in hexagon_gdf and sum the values within the current polygon
    for index, polygon in hexagon_train.iterrows():
        
        # Filter rows in taxi_gdf to only include points within the current polygon
        filtered_train = train_gdf[train_gdf['geometry'].within(polygon['geometry'])]
        
        # Sum the values of the current column within the filtered_taxi dataframe
        value_sum = filtered_train[i].sum()
        sums.append(value_sum)  # Append the sum to the list of sums
    
    # Add the sums as a new column in hexagon_gdf_copy
    hexagon_train[i] = sums

Processing column 'weekday_00'
Processing column 'weekday_05'
Processing column 'weekday_06'
Processing column 'weekday_07'
Processing column 'weekday_08'
Processing column 'weekday_09'
Processing column 'weekday_10'
Processing column 'weekday_11'
Processing column 'weekday_12'
Processing column 'weekday_13'
Processing column 'weekday_14'
Processing column 'weekday_15'
Processing column 'weekday_16'
Processing column 'weekday_17'
Processing column 'weekday_18'
Processing column 'weekday_19'
Processing column 'weekday_20'
Processing column 'weekday_21'
Processing column 'weekday_22'
Processing column 'weekday_23'
Processing column 'weekend_00'
Processing column 'weekend_05'
Processing column 'weekend_06'
Processing column 'weekend_07'
Processing column 'weekend_08'
Processing column 'weekend_09'
Processing column 'weekend_10'
Processing column 'weekend_11'
Processing column 'weekend_12'
Processing column 'weekend_13'
Processing column 'weekend_14'
Processing column 'weekend_15'
Processi

In [22]:
hexagon_train.head()

,h3_address,latitude,longitude,geometry,weekday_00,weekday_05,weekday_06,weekday_07,weekday_08,weekday_09,...,weekend_14,weekend_15,weekend_16,weekend_17,weekend_18,weekend_19,weekend_20,weekend_21,weekend_22,weekend_23
0,876520c18ffffff,1.290950,103.628706,"POLYGON ((103.64169 1.28738, 103.63852 1.30043...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,876526aeaffffff,1.289424,103.943911,"POLYGON ((103.95687 1.28585, 103.95372 1.29890...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,876520c82ffffff,1.415494,103.707587,"POLYGON ((103.72057 1.41192, 103.71741 1.42498...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,87652632effffff,1.430500,104.006687,"POLYGON ((104.01965 1.42692, 104.01650 1.43998...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,876520c93ffffff,1.460598,103.720903,"POLYGON ((103.73389 1.45702, 103.73072 1.47008...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
train_df['weekend_11'].sum()

1361628

In [24]:
hexagon_taxi.insert(0, 'h3_index', hexagon_taxi.index + 1)
hexagon_bus.insert(0, 'h3_index', hexagon_bus.index + 1)
hexagon_train.insert(0, 'h3_index', hexagon_train.index + 1)

In [25]:
hexagon_demand_first_5 = hexagon_bus.iloc[:, :5]

hexagon_demand_5_onwards = hexagon_bus.iloc[:, 5:] + hexagon_train.iloc[:, 5:]

hexagon_demand = pd.concat([hexagon_demand_first_5, hexagon_demand_5_onwards], axis=1)
hexagon_demand.head()

,h3_index,h3_address,latitude,longitude,geometry,weekday_00,weekday_01,weekday_02,weekday_03,weekday_04,...,weekend_14,weekend_15,weekend_16,weekend_17,weekend_18,weekend_19,weekend_20,weekend_21,weekend_22,weekend_23
0,1,876520c18ffffff,1.290950,103.628706,"POLYGON ((103.64169 1.28738, 103.63852 1.30043...",1,NaN,NaN,NaN,NaN,...,1010,922,1232,1669,1348,861,718,386,229,47
1,2,876526aeaffffff,1.289424,103.943911,"POLYGON ((103.95687 1.28585, 103.95372 1.29890...",0,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,3,876520c82ffffff,1.415494,103.707587,"POLYGON ((103.72057 1.41192, 103.71741 1.42498...",0,NaN,NaN,NaN,NaN,...,57,60,75,110,71,79,34,18,10,2
3,4,87652632effffff,1.430500,104.006687,"POLYGON ((104.01965 1.42692, 104.01650 1.43998...",0,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,5,876520c93ffffff,1.460598,103.720903,"POLYGON ((103.73389 1.45702, 103.73072 1.47008...",0,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [26]:
hexagon_final_first_5 = hexagon_taxi.iloc[:, :5]

hexagon_final_5_onwards = hexagon_demand.iloc[:, 5:] / (hexagon_taxi.iloc[:, 5:] + 1)

hexagon_final = pd.concat([hexagon_final_first_5, hexagon_final_5_onwards], axis=1)
hexagon_final = hexagon_final.fillna(0)
hexagon_final.head()

,h3_index,h3_address,latitude,longitude,geometry,weekday_00,weekday_01,weekday_02,weekday_03,weekday_04,...,weekend_14,weekend_15,weekend_16,weekend_17,weekend_18,weekend_19,weekend_20,weekend_21,weekend_22,weekend_23
0,1,876520c18ffffff,1.290950,103.628706,"POLYGON ((103.64169 1.28738, 103.63852 1.30043...",0.032258,0.0,0.0,0.0,0.0,...,252.5,131.714286,176.0,185.444444,168.5,71.75,35.9,35.090909,38.166667,11.75
1,2,876526aeaffffff,1.289424,103.943911,"POLYGON ((103.95687 1.28585, 103.95372 1.29890...",0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.00,0.0,0.000000,0.000000,0.00
2,3,876520c82ffffff,1.415494,103.707587,"POLYGON ((103.72057 1.41192, 103.71741 1.42498...",0.000000,0.0,0.0,0.0,0.0,...,28.5,60.000000,25.0,110.000000,35.5,79.00,34.0,6.000000,1.428571,1.00
3,4,87652632effffff,1.430500,104.006687,"POLYGON ((104.01965 1.42692, 104.01650 1.43998...",0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.00,0.0,0.000000,0.000000,0.00
4,5,876520c93ffffff,1.460598,103.720903,"POLYGON ((103.73389 1.45702, 103.73072 1.47008...",0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.00,0.0,0.000000,0.000000,0.00


In [48]:
hexagon_final['weekend_16'].sum()

31397.314176695305

#### Function to check the location of the hexagons

In [27]:
hexagon_final[hexagon_final['h3_index'] == 214]

,h3_index,h3_address,latitude,longitude,geometry,weekday_00,weekday_01,weekday_02,weekday_03,weekday_04,...,weekend_14,weekend_15,weekend_16,weekend_17,weekend_18,weekend_19,weekend_20,weekend_21,weekend_22,weekend_23
213,214,876520ca9ffffff,1.336037,103.642011,"POLYGON ((103.65500 1.33246, 103.65183 1.34552...",1.339286,0.0,0.0,0.0,0.0,...,1157.8,768.25,1850.0,1586.571429,1144.888889,780.0,441.9375,551.555556,333.6,81.0


In [28]:
#geo_subset = hexagon_final.loc[hexagon_final['weekend_18'].idxmax(), ['geometry']]
geo_subset = hexagon_final[hexagon_final['h3_index'] == 243]['geometry']
geo_subset

242    POLYGON ((104.00908 1.32018, 104.00594 1.33323...
Name: geometry, dtype: geometry

In [29]:
hexagon_final.to_csv('assets/processed_data/modeling_final.csv')

#### Find the top 5 areas with highest taxi available during weekday office hours

In [30]:
# Assuming hexagon_taxi is your DataFrame and has been loaded previously
top_3_rows = hexagon_taxi.nlargest(6, 'weekday_08')[['h3_index', 'weekday_08']]
print(top_3_rows)

     h3_index  weekday_08
136       137        3068
187       188        1234
226       227        1233
184       185        1042
134       135         946
94         95         938


In [53]:
import folium
from shapely.geometry import Polygon

# Assuming geo_subset is a DataFrame containing geometries and other data
geo_subset = hexagon_final[(hexagon_final['h3_index'] == 137) | (hexagon_final['h3_index'] == 95) | (hexagon_final['h3_index'] == 188)
| (hexagon_final['h3_index'] == 227) | (hexagon_final['h3_index'] == 185)]

# Create a Folium map centered at a location of your choice
map_center = [1.3521, 103.8198]  # Provide latitude and longitude of the map center
mymap = folium.Map(location=map_center, zoom_start=12, tiles='cartodbpositron')

# Iterate through each geometry in geo_subset['geometry']
for index, row in geo_subset.iterrows():
    # Extract the coordinates of the hexagon
    coordinates = row['geometry'].exterior.coords
    
    # Create a list of coordinates
    coord_list = [[coord[1], coord[0]] for coord in coordinates]
    
    # Create a Folium polygon using the extracted coordinates
    folium.Polygon(locations=coord_list, color='blue', fill=True, fill_color='blue').add_to(mymap)

# Display the map
mymap

In [32]:
# Assuming hexagon_taxi is your DataFrame and has been loaded previously
top_4_rows_1800 = hexagon_taxi.nlargest(6, 'weekday_18')[['h3_index', 'weekday_18']]
print(top_4_rows_1800)

     h3_index  weekday_18
136       137        4616
134       135        1607
187       188        1434
198       199        1374
94         95        1106
226       227        1050


In [54]:
import folium
from shapely.geometry import Polygon

# Assuming geo_subset is a DataFrame containing geometries and other data
geo_subset = hexagon_final[(hexagon_final['h3_index'] == 137) | (hexagon_final['h3_index'] == 188) | (hexagon_final['h3_index'] == 199)
| (hexagon_final['h3_index'] == 95) | (hexagon_final['h3_index'] == 227)]

# Create a Folium map centered at a location of your choice
map_center = [1.3521, 103.8198]  # Provide latitude and longitude of the map center
mymap = folium.Map(location=map_center, zoom_start=12, tiles='cartodbpositron')

# Iterate through each geometry in geo_subset['geometry']
for index, row in geo_subset.iterrows():
    # Extract the coordinates of the hexagon
    coordinates = row['geometry'].exterior.coords
    
    # Create a list of coordinates
    coord_list = [[coord[1], coord[0]] for coord in coordinates]
    
    # Create a Folium polygon using the extracted coordinates
    folium.Polygon(locations=coord_list, color='blue', fill=True, fill_color='blue').add_to(mymap)

# Display the map
mymap

#### Find the top 5 areas with highest taxi available during weekend lunch and dinner

In [34]:
# Assuming hexagon_taxi is your DataFrame and has been loaded previously
top_3_rows_weekend_11 = hexagon_taxi.nlargest(6, 'weekend_11')[['h3_index', 'weekend_11']]
print(top_3_rows_weekend_11)

     h3_index  weekend_11
136       137        1374
134       135         661
198       199         647
46         47         631
33         34         600
87         88         578


In [55]:
import folium
from shapely.geometry import Polygon

# Assuming geo_subset is a DataFrame containing geometries and other data
geo_subset = hexagon_final[(hexagon_final['h3_index'] == 137) | (hexagon_final['h3_index'] == 199) | (hexagon_final['h3_index'] == 47)
| (hexagon_final['h3_index'] == 34) | (hexagon_final['h3_index'] == 88)]

# Create a Folium map centered at a location of your choice
map_center = [1.3521, 103.8198]  # Provide latitude and longitude of the map center
mymap = folium.Map(location=map_center, zoom_start=12, tiles='cartodbpositron')

# Iterate through each geometry in geo_subset['geometry']
for index, row in geo_subset.iterrows():
    # Extract the coordinates of the hexagon
    coordinates = row['geometry'].exterior.coords
    
    # Create a list of coordinates
    coord_list = [[coord[1], coord[0]] for coord in coordinates]
    
    # Create a Folium polygon using the extracted coordinates
    folium.Polygon(locations=coord_list, color='blue', fill=True, fill_color='blue').add_to(mymap)

# Display the map
mymap

In [36]:
# Assuming hexagon_taxi is your DataFrame and has been loaded previously
top_3_rows_weekend_18 = hexagon_taxi.nlargest(6, 'weekend_18')[['h3_index', 'weekend_18']]
print(top_3_rows_weekend_18)

     h3_index  weekend_18
136       137        1089
198       199         477
187       188         438
226       227         421
134       135         394
46         47         389


In [56]:
import folium
from shapely.geometry import Polygon

# Assuming geo_subset is a DataFrame containing geometries and other data
geo_subset = hexagon_final[(hexagon_final['h3_index'] == 137) | (hexagon_final['h3_index'] == 199) | (hexagon_final['h3_index'] == 188)
| (hexagon_final['h3_index'] == 227) | (hexagon_final['h3_index'] == 47)]

# Create a Folium map centered at a location of your choice
map_center = [1.3521, 103.8198]  # Provide latitude and longitude of the map center
mymap = folium.Map(location=map_center, zoom_start=12, tiles='cartodbpositron')

# Iterate through each geometry in geo_subset['geometry']
for index, row in geo_subset.iterrows():
    # Extract the coordinates of the hexagon
    coordinates = row['geometry'].exterior.coords
    
    # Create a list of coordinates
    coord_list = [[coord[1], coord[0]] for coord in coordinates]
    
    # Create a Folium polygon using the extracted coordinates
    folium.Polygon(locations=coord_list, color='blue', fill=True, fill_color='blue').add_to(mymap)

# Display the map
mymap

#### Find the top 5 areas with highest taxi available during weekday office hours|

In [38]:
# Assuming hexagon_taxi is your DataFrame and has been loaded previously
top_5_rows_weekday_08 = hexagon_demand.nlargest(5, 'weekday_08')[['h3_index', 'weekday_08']]
print(top_5_rows_weekday_08)

     h3_index  weekday_08
181       182      716935
94         95      456975
223       224      442522
229       230      439908
187       188      428286


In [57]:
import folium
from shapely.geometry import Polygon

# Assuming geo_subset is a DataFrame containing geometries and other data
geo_subset = hexagon_final[(hexagon_final['h3_index'] == 182) | (hexagon_final['h3_index'] == 95) | (hexagon_final['h3_index'] == 224)
| (hexagon_final['h3_index'] == 230) | (hexagon_final['h3_index'] == 188)]

# Create a Folium map centered at a location of your choice
map_center = [1.3521, 103.8198]  # Provide latitude and longitude of the map center
mymap = folium.Map(location=map_center, zoom_start=12, tiles='cartodbpositron')

# Iterate through each geometry in geo_subset['geometry']
for index, row in geo_subset.iterrows():
    # Extract the coordinates of the hexagon
    coordinates = row['geometry'].exterior.coords
    
    # Create a list of coordinates
    coord_list = [[coord[1], coord[0]] for coord in coordinates]
    
    # Create a Folium polygon using the extracted coordinates
    folium.Polygon(locations=coord_list, color='blue', fill=True, fill_color='blue').add_to(mymap)

# Display the map
mymap

In [40]:
# Assuming hexagon_taxi is your DataFrame and has been loaded previously
top_5_rows_weekday_18 = hexagon_demand.nlargest(5, 'weekday_18')[['h3_index', 'weekday_18']]
print(top_5_rows_weekday_18)

     h3_index  weekday_18
198       199     1104321
226       227      894084
46         47      813100
190       191      544355
181       182      487792


In [58]:
import folium
from shapely.geometry import Polygon

# Assuming geo_subset is a DataFrame containing geometries and other data
geo_subset = hexagon_final[(hexagon_final['h3_index'] == 199) | (hexagon_final['h3_index'] == 227) | (hexagon_final['h3_index'] == 47)
| (hexagon_final['h3_index'] == 191) | (hexagon_final['h3_index'] == 182)]

# Create a Folium map centered at a location of your choice
map_center = [1.3521, 103.8198]  # Provide latitude and longitude of the map center
mymap = folium.Map(location=map_center, zoom_start=12, tiles='cartodbpositron')

# Iterate through each geometry in geo_subset['geometry']
for index, row in geo_subset.iterrows():
    # Extract the coordinates of the hexagon
    coordinates = row['geometry'].exterior.coords
    
    # Create a list of coordinates
    coord_list = [[coord[1], coord[0]] for coord in coordinates]
    
    # Create a Folium polygon using the extracted coordinates
    folium.Polygon(locations=coord_list, color='blue', fill=True, fill_color='blue').add_to(mymap)

# Display the map
mymap

#### Find the top 5 areas with highest taxi available during weekend lunch and dinner

In [42]:
# Assuming hexagon_taxi is your DataFrame and has been loaded previously
top_5_rows_weekend_11 = hexagon_demand.nlargest(5, 'weekend_11')[['h3_index', 'weekend_11']]
print(top_5_rows_weekend_11)

     h3_index  weekend_11
46         47      151835
181       182      150060
226       227      112178
187       188      112096
139       140      110923


In [59]:
import folium
from shapely.geometry import Polygon

# Assuming geo_subset is a DataFrame containing geometries and other data
geo_subset = hexagon_final[(hexagon_final['h3_index'] == 47) | (hexagon_final['h3_index'] == 182) | (hexagon_final['h3_index'] == 227)
| (hexagon_final['h3_index'] == 188) | (hexagon_final['h3_index'] == 140)]

# Create a Folium map centered at a location of your choice
map_center = [1.3521, 103.8198]  # Provide latitude and longitude of the map center
mymap = folium.Map(location=map_center, zoom_start=12, tiles='cartodbpositron')

# Iterate through each geometry in geo_subset['geometry']
for index, row in geo_subset.iterrows():
    # Extract the coordinates of the hexagon
    coordinates = row['geometry'].exterior.coords
    
    # Create a list of coordinates
    coord_list = [[coord[1], coord[0]] for coord in coordinates]
    
    # Create a Folium polygon using the extracted coordinates
    folium.Polygon(locations=coord_list, color='blue', fill=True, fill_color='blue').add_to(mymap)

# Display the map
mymap

In [44]:
# Assuming hexagon_taxi is your DataFrame and has been loaded previously
top_5_rows_weekend_18 = hexagon_demand.nlargest(5, 'weekend_18')[['h3_index', 'weekend_18']]
print(top_5_rows_weekend_18)

     h3_index  weekend_18
46         47      262573
198       199      204682
181       182      152473
226       227      134654
33         34      132608


In [60]:
import folium
from shapely.geometry import Polygon

# Assuming geo_subset is a DataFrame containing geometries and other data
geo_subset = hexagon_final[(hexagon_final['h3_index'] == 47) | (hexagon_final['h3_index'] == 199) | (hexagon_final['h3_index'] == 182)
| (hexagon_final['h3_index'] == 227) | (hexagon_final['h3_index'] == 34)]

# Create a Folium map centered at a location of your choice
map_center = [1.3521, 103.8198]  # Provide latitude and longitude of the map center
mymap = folium.Map(location=map_center, zoom_start=12, tiles='cartodbpositron')

# Iterate through each geometry in geo_subset['geometry']
for index, row in geo_subset.iterrows():
    # Extract the coordinates of the hexagon
    coordinates = row['geometry'].exterior.coords
    
    # Create a list of coordinates
    coord_list = [[coord[1], coord[0]] for coord in coordinates]
    
    # Create a Folium polygon using the extracted coordinates
    folium.Polygon(locations=coord_list, color='blue', fill=True, fill_color='blue').add_to(mymap)

# Display the map
mymap

In [46]:
import folium
from shapely.geometry import Polygon

# Assuming geo_subset is a DataFrame containing geometries and other data
geo_subset = hexagon_final[(hexagon_final['h3_index'] == 243) | (hexagon_final['h3_index'] == 99) | (hexagon_final['h3_index'] == 214)]

# Create a Folium map centered at a location of your choice
map_center = [1.3521, 103.8198]  # Provide latitude and longitude of the map center
mymap = folium.Map(location=map_center, zoom_start=12)

# Iterate through each geometry in geo_subset['geometry']
for index, row in geo_subset.iterrows():
    # Extract the coordinates of the hexagon
    coordinates = row['geometry'].exterior.coords
    
    # Create a list of coordinates
    coord_list = [[coord[1], coord[0]] for coord in coordinates]
    
    # Create a Folium polygon using the extracted coordinates
    polygon = folium.Polygon(locations=coord_list, color='blue', fill=True, fill_color='blue')
    polygon.add_to(mymap)
    
    # Add value as text overlay inside the hexagon
    value = 10  # Assuming 'value' is the column containing the value you want to display
    folium.Marker(location=row['geometry'].centroid.coords[0][::-1], icon=folium.DivIcon(html=f"<div style='font-size: 10pt'>{value}</div>")).add_to(mymap)

# Display the map
mymap